In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../data/traditional.csv")
df.head()

,text,target
0,The VIRSA (Visible Infrared Survey Telescope A...,real
1,China\nThe goal of this project involves achie...,fake
2,Scientists can learn about how galaxies form a...,real
3,China\nThe study suggests that multiple star s...,fake
4,Dinosaur Rex was excited about his new toy set...,fake


In [4]:
test = pd.read_csv('../data/test_processed.csv')
test.head()

,sample_id,file_1.txt_context,file_2.txt_context
0,article_0000,"""Music"" Music music music Music music Music mu...",Since its launch on Paranal observatory's Very...
1,article_0001,underground exploration on SN's birth has prov...,SN 1987A provides valuable insights as newer o...
2,article_0002,This research aimed to understand how star sha...,ChromeDriver music player\nThis study focused ...
3,article_0003,Using OmegaCAM's wide field capabilities spann...,"greek translation :\nvazhi (megaCAM), territor..."
4,article_0004,AssemblyCulture AssemblyCulture AssemblyCultur...,XClass is software tool that helps astronomers...
